# 데이터분석프로그래밍 프로젝트
## 컴퓨터공학과 201911771 임영서

### 데이터: 아마존 패션 리뷰 데이터  
데이터 다운로드: http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz  
데이터 다운로드: http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_AMAZON_FASHION.json.gz

### json.gz 파일을 열기 위한 작업

In [9]:
import pandas as pd

In [3]:
# https://jmcauley.ucsd.edu/data/amazon/
# 위 주소에서 파일 여는 방법에 대해 알아봄

import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient = 'index')

In [16]:
meta = getDF('./meta_AMAZON_FASHION.json.gz')
meta.shape

(186637, 16)

In [10]:
meta

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6855,NaN,GUESS,NaN,NaN,5 star,B002Y2F192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58116,NaN,Favebridal,[Package Dimensions:\n \n10...,"5,143,860inClothing,ShoesJewelry(",5 star,B00NBF3FDK,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9345,NaN,NaN,[Shipping Information:\n \n...,"3,376,854inClothing,ShoesJewelry(",NA,B004U7OHV2,NaN,NaN,NaN,$5.99 - $8.99,NaN,NaN,NaN,NaN,NaN,NaN
144648,NaN,NaN,[Package Dimensions:\n \n9....,"1,385,785inClothing,ShoesJewelry(",5 star,B01AWWE8A8,NaN,NaN,NaN,$13.95,NaN,"[B07C91QJW2, B079NQ5XKW, B07CQ759K6, B00I77CGJ...",NaN,NaN,NaN,NaN


In [11]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186637 entries, 0 to 169249
Data columns (total 16 columns):
title              186632 non-null object
brand              139957 non-null object
feature            123875 non-null object
rank               180222 non-null object
date               185001 non-null object
asin               186637 non-null object
imageURL           132017 non-null object
imageURLHighRes    132017 non-null object
description        15869 non-null object
price              17799 non-null object
also_view          11595 non-null object
also_buy           21642 non-null object
fit                4831 non-null object
details            885 non-null object
similar_item       317 non-null object
tech1              97 non-null object
dtypes: object(16)
memory usage: 24.2+ MB


### 데이터의 생김새
- title: 제목
- brand: 상품의 브랜드
- feature: 상품의 특징들
- rank: 해당 카테고리에서 몇 위까지 했는지
- date: ?
- asin: 상품 고유 코드
- imageURL: 이미지 URL
- imageURLHighRes: 고화질 이미지 URL
- description: 상품의 설명
- price: 가격
- also_view: 같이 본 상품
- also_buy: 같이 산 상품
- fit: ?
- details: 상품의 디테일
- similar_item: 비슷한 상품
- tech1: ?

### 위 데이터로 dashboard를 만들 예정!  
### 사용할 데이터
- brand
- imageURLHighRes
- price

### 목표
브랜드와 가격대를 입력하면 해당하는 상품들의 사진을 보여준다.

In [126]:
data = meta[['brand', 'imageURLHighRes', 'price']]
data = data.dropna()
data

,brand,imageURLHighRes,price
17,Roar Carbon,[https://images-na.ssl-images-amazon.com/image...,$14.99
18,Shimmer Anne Shine,[https://images-na.ssl-images-amazon.com/image...,$6.99
69,Buxton,[https://images-na.ssl-images-amazon.com/image...,$16.95
331,Silver Insanity,[https://images-na.ssl-images-amazon.com/image...,$44.66
388,Silver Forest,[https://images-na.ssl-images-amazon.com/image...,$23.00
...,...,...,...
186563,LenzFlip,[https://images-na.ssl-images-amazon.com/image...,$30.00
186579,Stetson,[https://images-na.ssl-images-amazon.com/image...,$99.99
186589,Ivysport,[https://images-na.ssl-images-amazon.com/image...,$16.95
186603,IK COLOURING,[https://images-na.ssl-images-amazon.com/image...,$52.50


# 이제 대쉬보드를 만들어보자!

**추가**  
+) 주피터 노트북에서 대쉬보드를 만드는 방법을 몰라서.. 파이썬 파일로 만들었답니다~!

In [134]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig)
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 98] Address already in use